In [162]:
import pandas as pd

In [163]:
df = pd.read_csv('Cyber1_train.csv')

In [164]:
df.head()

,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,151143,0.000003,udp,-,INT,2,0,104,0,333333.321500,...,1,2,0,0,0,1,3,0,Fuzzers,1
1,80153,2.308886,tcp,-,FIN,10,8,564,354,7.362858,...,1,1,0,0,0,1,1,0,Reconnaissance,1
2,164542,0.000003,udp,dns,INT,2,0,114,0,333333.321500,...,17,17,0,0,0,18,17,0,Generic,1
3,32390,0.004722,tcp,-,FIN,18,18,3960,2456,7412.113355,...,1,4,0,0,0,7,7,0,Normal,0
4,161063,0.000002,udp,dns,INT,2,0,114,0,500000.001300,...,16,29,0,0,0,16,29,0,Generic,1


In [165]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112217 entries, 0 to 112216
Data columns (total 45 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   id                 112217 non-null  int64  
 1   dur                112217 non-null  float64
 2   proto              112217 non-null  object 
 3   service            112217 non-null  object 
 4   state              112217 non-null  object 
 5   spkts              112217 non-null  int64  
 6   dpkts              112217 non-null  int64  
 7   sbytes             112217 non-null  int64  
 8   dbytes             112217 non-null  int64  
 9   rate               112217 non-null  float64
 10  sttl               112217 non-null  int64  
 11  dttl               112217 non-null  int64  
 12  sload              112217 non-null  float64
 13  dload              112217 non-null  float64
 14  sloss              112217 non-null  int64  
 15  dloss              112217 non-null  int64  
 16  si

In [166]:
df['attack_cat'].value_counts()

attack_cat
Normal            35840
Generic           25585
Exploits          21514
Fuzzers           11593
DoS                7837
Reconnaissance     6655
Analysis           1257
Backdoor           1133
Shellcode           722
Worms                81
Name: count, dtype: int64

In [167]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

In [168]:
df['attack_cat'] = le.fit_transform(df['attack_cat'])

In [169]:
df['proto'] = le.fit_transform(df['proto'])
df['service'] = le.fit_transform(df['service'])
df['state'] = le.fit_transform(df['state'])

In [173]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

In [174]:
df_scaled

array([[ 1.25552432, -0.20911086,  0.42020499, ..., -0.5676767 ,
        -0.12655666, -0.42596238],
       [-0.14726096,  0.14997587,  0.15173381, ..., -0.75364391,
        -0.12655666,  1.4883947 ],
       [ 1.52029288, -0.20911086,  0.42020499, ...,  0.7340938 ,
        -0.12655666,  0.21215665],
       ...,
       [-0.95632759, -0.20764551,  0.15173381, ..., -0.5676767 ,
        -0.12655666,  0.85027567],
       [ 1.63703686, -0.20911055,  0.42020499, ...,  1.29199544,
        -0.12655666,  0.21215665],
       [-0.1796086 , -0.03344323,  0.15173381, ..., -0.6606603 ,
        -0.12655666, -0.42596238]])

In [175]:
y=df['label']

In [176]:
X = df.drop('label', axis = 1)
X.head()
df_scaled = scaler.fit_transform(X)

In [195]:
from sklearn.decomposition import PCA

pca = PCA(n_components = 3)

In [196]:
df_pca = pca.fit_transform(df_scaled)

In [199]:
df_pca = pd.DataFrame(data=df_pca, columns=['PC1', 'PC2', 'PC3'])

In [200]:
df_pca.head()

,PC1,PC2,PC3
0,-1.359233,-2.059613,-0.100073
1,3.420552,0.816738,-2.183103
2,-4.396629,1.005340,-0.253752
3,1.412969,-0.473506,1.592930
4,-5.286594,1.491276,-0.274324


In [201]:
y

0         1
1         1
2         1
3         0
4         1
         ..
112212    0
112213    1
112214    0
112215    1
112216    1
Name: label, Length: 112217, dtype: int64

In [202]:
df_pca.shape

(112217, 3)

In [203]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_pca, y, test_size = 0.3, random_state = 32)

In [211]:
from sklearn.svm import SVC

model = SVC(kernel = 'rbf', C = 1)

In [212]:
model.fit(X_train,y_train)
model.score(X_test, y_test)

0.9001960434860097

In [206]:
from sklearn.metrics import accuracy_score

y_pred = model.predict(X_test)

In [207]:
print(accuracy_score(y_test, y_pred))

0.9206023881661023


In [208]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      0.76      0.86     10837
           1       0.90      1.00      0.94     22829

    accuracy                           0.92     33666
   macro avg       0.94      0.88      0.90     33666
weighted avg       0.93      0.92      0.92     33666

